title and description

In [ ]:
import os
#os.environ['create_image']='True'
#os.environ['repository']='romeokienzler'
os.environ['install_requirements']='True'


In [ ]:
import os
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-38
    RUN pip install nvflare==2.0.16
    RUN pip install tensorflow==2.9.1
    ADD nvflare.ipynb /
    ENTRYPOINT ["ipython","/nvflare.ipynb"]
    """
    with open("Dockerfile", "w") as text_file: 
        text_file.write(docker_file)

    !docker build -t claimed-train-nvflare .
    !docker tag claimed-train-nvflare `echo $repository`/claimed-train-nvflare
    !docker push `echo $repository`/claimed-train-nvflare
elif bool(os.environ.get('install_requirements',False)):
    !wget https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt
    !pip install -r requirements.txt
    !pip install yolo5==0.0.1 torch==1.11.0


In [ ]:
import os
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50V2 # , MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model
from claimed_utils import unzip, zipdir
import os.path
import glob
from io import BytesIO
from minio import Minio
from ansible.module_utils.parsing.convert_bool import boolean

In [ ]:
# @dependency codait_utils.ipynb
# @param image_shape shape the images shall be scaled to and the
# model then will accept
# @param model zip file name
# @param data zip file name
# @param model folder name
# @param data folder name
# @param epochs number of epochs to train
# @param checkpoint activate checkpointing
# @param checkpoint_ip minio endpoint
# @param checkpoint_user minio user
# @param checkpoint_pass minio pw
# @param checkpoint_bucket minio bucket
# @returns model zip file

In [ ]:
image_shape = os.environ.get('image_shape', '400,400')
model_zip = os.environ.get('model_zip', 'model.zip')
data_zip = os.environ.get('data_zip', 'data.zip')
model_folder = os.environ.get('model', 'model')
data = os.environ.get('data', 'data')
epochs = int(os.environ.get('epochs', 1))
checkpoint = boolean(os.environ.get('checkpoint', False))
checkpoint_ip = os.environ.get('checkpoint_ip')
checkpoint_user = os.environ.get('checkpoint_user', 'minio')
checkpoint_pass = os.environ.get('checkpoint_pass', 'minio123')
checkpoint_bucket = os.environ.get('checkpoint_bucket', 'checkpoint')

In [ ]:
!wget https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg


In [ ]:
import yolov5

# model
model = yolov5.load('yolov5s')

# image
img = 'https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg'

# inference
results = model(img)

# inference with larger input size
results = model(img, size=1280)

# inference with test time augmentation
results = model(img, augment=True)

# show results
results.show()

# save results
results.save(save_dir='results/')

In [1]:
import yolov5

# model
model = yolov5.load('yolov5s')

In [4]:
# image
img = 'https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg'

# inference
results = model(img, augment=True)


AttributeError: 'DetectMultiBackend' object has no attribute 'input_details'

In [ ]:
from yolov5 import YOLOv5
import torch 

torch.load(map_location=torch.device('cpu'))
# set model params
model_path = "yolov5/weights/yolov5s.pt" # it automatically downloads yolov5s model to given path
device = "cuda" # or "cpu"

# init yolov5 model
yolov5 = YOLOv5(model_path, device)

# load images
image1 = 'https://github.com/ultralytics/yolov5/raw/master/data/images/zidane.jpg'
image2 = 'https://github.com/ultralytics/yolov5/blob/master/data/images/bus.jpg'

# perform inference
results = yolov5.predict(image1)

# perform inference with larger input size
results = yolov5.predict(image1, size=1280)

# perform inference with test time augmentation
results = yolov5.predict(image1, augment=True)

# perform inference on multiple images
results = yolov5.predict([image1, image2], size=1280, augment=True)

# show detection bounding boxes on image
results.show()

# save results into "results/" folder
results.save(save_dir='results/')

In [ ]:
!pip install torch==1.11.0